In [11]:
from langchain_ollama import OllamaLLM
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import OllamaEmbeddings

In [18]:
# %%
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import os


In [20]:
# Cargar y procesar el CSV
loader = CSVLoader(file_path="books.csv")
data = loader.load()
        
# Dividir los documento
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
splits = text_splitter.split_documents(data)
        
# Crear embeddings y vectorstore
embeddings = OllamaEmbeddings(
    model="qwen2.5",
    base_url="http://192.168.0.25:11434"
)
vector = FAISS.from_documents(splits, embeddings)
retriever = vector.as_retriever(earch_kwargs={"k": 1})
llm = OllamaLLM(
    model="qwen2.5",
    base_url="http://192.168.0.25:11434",
    num_gpu_layers=90,       # Aceleración Metal máxima
    num_ctx=1536,            # Balance memoria/contexto
    temperature=0.5,
)

In [24]:
# %%
# Definir el prompt para recomendaciones
template = """Eres un asistente literario experto en recomendar libros. 
Basándote en el contexto proporcionado, recomienda libros que coincidan con los gustos del usuario.

Contexto:
{context}

Consulta del usuario: {query}

Proporciona:
1. 1 recomendaciones de libros con título y autor
2. Breve explicación de por qué cada libro podría gustarle
3. Género y año de publicación
4. Usa un tono amigable y entusiasta

Formato de respuesta:
### Recomendaciones para [resumen de la consulta]

1. **Título del Libro** - Autor (Año)
   - Género: [género]
   - ¿Por qué?: [explicación]
   
... (repetir para cada recomendación)"""

prompt = ChatPromptTemplate.from_template(template)

# Crear cadena de recomendación
recommendation_chain = (
    {"context": retriever, "query": lambda x: x["query"]}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
# %%
# Función para probar recomendaciones
def test_recommendation(query):
    print(f"\n\033[1mConsulta:\033[0m {query}")
    print("\n\033[1mRecomendaciones:\033[0m")
    result = recommendation_chain.invoke({"query": query})
    print(result)

# %% [markdown]
"""
## Pruebas del sistema
Ejecuta las siguientes celdas para probar diferentes consultas
"""

# %%
test_recommendation("Libros clásicos que todo el mundo debería leer")



Consulta: Libros clásicos que todo el mundo debería leer

Recomendaciones:
### Recomendaciones para libros clásicos que todo el mundo debería leer

1. **Cien años de soledad** - Gabriel García Márquez (1967)
   - Género: realismo mágico | ficción literaria | saga familiar
   - ¿Por qué?: "Cien años de soledad" es una obra maestra que combina la realidad y el fantástico en un relato épico sobre la familia Buendía. García Márquez nos transporta a un pueblo mágico llamado Macondo, donde los eventos se entrelazan con una magia realista que cautivará tu imaginación.

2. **1984** - George Orwell (1949)
   - Género: distopía | ficción política | clásico
   - ¿Por qué?: "1984" es un retrato sombrío pero indispensable de un futuro totalitario bajo el control omnipresente del Partido. Orwell nos muestra cómo la verdad y la individualidad son suprimidas, lo que hace que esta obra sea más relevante hoy que nunca.

Estos libros no solo son clásicos, sino también fundamentales para entender la lite